In [1]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from matplotlib.gridspec import GridSpec
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from warnings import filterwarnings
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, silhouette_samples
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, dendrogram
from factor_analyzer import FactorAnalyzer
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import ElasticNet
from itertools import combinations
from kmodes.kprototypes import KPrototypes
from collections import Counter
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFE
### so that u dont have warnings
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.cluster import DBSCAN
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

C:\Users\Yehan Perera\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df_1 = pd.read_csv("zomato.csv")

In [3]:
def preprocessing_initial(data):
    ### Remove unwanted columns 
    col_to_remove = ['url','address','phone','reviews_list','menu_item']
    df = data.drop(columns=col_to_remove)
    
    # remove duplicates
    df.drop_duplicates(inplace=True)
    

    def fun(x):
        x_new = x.split(',')
        return x_new[0]+x_new[1]
    df['approx_cost(for two people)']=df['approx_cost(for two people)'].apply(lambda x: fun(x) if ',' in str(x) else x)
    df['approx_cost(for two people)'] = pd.to_numeric(df['approx_cost(for two people)'])
    
    
    # rate 
    df['rate'] = df['rate'].apply(lambda x : str(x).split('/')[0])
    ls = [float(i) for i in df['rate'] if (i!='NEW') & (i!='nan') & (i!='-')]
    mean_rate = round(np.mean(ls),1)
    df['rate'] = df['rate'].apply(lambda x: mean_rate if (x=='NEW') | (x=='-') | (x=='nan') else x)
    df['rate'] = pd.to_numeric(df['rate'])
    ## Remove the missing values of response 
    df.dropna(subset=['approx_cost(for two people)'], inplace=True)
    
     #########   categorical variables 
    vars_to_impute = ['location','rest_type','dish_liked','cuisines']
    for col in vars_to_impute:
        df.dropna(subset=[col],inplace=True)
        
        
    ## Split the data 
    X= df.drop(columns=['approx_cost(for two people)'])
    Y = df['approx_cost(for two people)']
   
    return   X ,Y 
    

In [4]:
X , Y = preprocessing_initial(df_1)

In [5]:
from category_encoders import TargetEncoder

In [6]:
from sklearn.preprocessing import OneHotEncoder
# Define your target encoder columns
target_encoder_cols = ['name', 'dish_liked', 'cuisines', 'location', 'rest_type', 'listed_in(city)', 'listed_in(type)', 'online_order', 'book_table']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Perform frequency encoding for each column
for col in target_encoder_cols:
    target_encoder = TargetEncoder()

    # Fit and transform the target encoder on the training data
    X_train[target_encoder_cols] = target_encoder.fit_transform(X_train[target_encoder_cols], y_train)

    # Transform the test data using the target encoder fitted on the training data
    X_test[target_encoder_cols] = target_encoder.transform(X_test[target_encoder_cols])


In [7]:
from sklearn.ensemble import IsolationForest
random_seed = 42
clf = IsolationForest(contamination=0.2, random_state=random_seed)
clf.fit(X_train)
predictions = clf.predict(X_train)

X_train['cluster_label'] = predictions

X_train_1 = X_train[X_train['cluster_label']== -1]
X_train_2 = X_train[X_train['cluster_label']== 1]

indexes_1 = X_train_1.index
indexes_2 = X_train_2.index


y_train_1 = y_train.loc[indexes_1]
y_train_2 = y_train.loc[indexes_2]







In [8]:
x_train_cluster_1 , x_test_cluster_1 , y_train_cluster_1 , y_test_cluster_1 = train_test_split(X_train_1,y_train_1,test_size=0.2,random_state=42)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Create Pipeline with feature selection and RandomForest
pipeline_rf = Pipeline([
    ('feature_selection', SelectFromModel(RandomForestRegressor())),
    ('rf', RandomForestRegressor())
])

# Define parameter grid for hyperparameter tuning for RandomForest
rf_params = {
    'rf__n_estimators': [50, 100, 200, 300],
    'rf__max_depth': [None, 5, 10, 20],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__max_features': ['auto', 'sqrt', 'log2']
}

# Assuming x_train_cluster_1, y_train_cluster_1, x_test_cluster_1, y_test_cluster_1 are defined

# Perform GridSearchCV for RandomForest
grid_search_rf = GridSearchCV(pipeline_rf, rf_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV for RandomForest
grid_search_rf.fit(x_train_cluster_1, y_train_cluster_1)

# Get the best model for RandomForest
best_model_rf_1 = grid_search_rf.best_estimator_

# Get the selected features for RandomForest
selected_features_rf = x_train_cluster_1.columns[best_model_rf_1.named_steps['feature_selection'].get_support()]

# Evaluate the best model for RandomForest
mape_train_rf = np.mean(np.abs((y_train_cluster_1 - best_model_rf_1.predict(x_train_cluster_1)) / y_train_cluster_1)) * 100
mape_test_rf = np.mean(np.abs((y_test_cluster_1 - best_model_rf_1.predict(x_test_cluster_1)) / y_test_cluster_1)) * 100

# Print the results
print("Selected Features (RandomForest):", selected_features_rf)
print("Mean Absolute Percentage Error on Training set (RandomForest):", mape_train_rf)
print("Mean Absolute Percentage Error on Testing set (RandomForest):", mape_test_rf)
print("Best Hyperparameters (RandomForest):", grid_search_rf.best_params_)



Selected Features (RandomForest): Index(['name', 'rest_type', 'dish_liked'], dtype='object')
Mean Absolute Percentage Error on Training set (RandomForest): 0.6968742631637074
Mean Absolute Percentage Error on Testing set (RandomForest): 1.8139797025002327
Best Hyperparameters (RandomForest): {'rf__max_depth': 20, 'rf__max_features': 'log2', 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 200}


In [11]:
# Predictions for Cluster 1 using Random Forest
y_train_pred_rf_cluster_1 = best_model_rf_1.predict(x_train_cluster_1)
y_test_pred_rf_cluster_1 = best_model_rf_1.predict(x_test_cluster_1)

# Calculate RMSE for Cluster 1 using Random Forest
rmse_train_rf_cluster_1 = np.sqrt(mean_squared_error(y_train_cluster_1, y_train_pred_rf_cluster_1))
rmse_test_rf_cluster_1 = np.sqrt(mean_squared_error(y_test_cluster_1, y_test_pred_rf_cluster_1))

# Mean Absolute Percentage Error for Cluster 1 using Random Forest
mape_train_rf_cluster_1 = np.mean(np.abs((y_train_cluster_1 - y_train_pred_rf_cluster_1) / y_train_cluster_1)) * 100
mape_test_rf_cluster_1 = np.mean(np.abs((y_test_cluster_1 - y_test_pred_rf_cluster_1) / y_test_cluster_1)) * 100

print("Selected Features (Random Forest) for Cluster 1:", selected_features_rf)
print("Mean Absolute Percentage Error on Training set (Random Forest) for Cluster 1:", mape_train_rf_cluster_1)
print("Mean Absolute Percentage Error on Testing set (Random Forest) for Cluster 1:", mape_test_rf_cluster_1)
print("Root Mean Squared Error on Training set (Random Forest) for Cluster 1:", rmse_train_rf_cluster_1)
print("Root Mean Squared Error on Testing set (Random Forest) for Cluster 1:", rmse_test_rf_cluster_1)
print(grid_search_rf.best_params_)


Selected Features (Random Forest) for Cluster 1: Index(['name', 'rest_type', 'dish_liked'], dtype='object')
Mean Absolute Percentage Error on Training set (Random Forest) for Cluster 1: 0.6968742631637074
Mean Absolute Percentage Error on Testing set (Random Forest) for Cluster 1: 1.8139797025002327
Root Mean Squared Error on Training set (Random Forest) for Cluster 1: 31.141131460264415
Root Mean Squared Error on Testing set (Random Forest) for Cluster 1: 85.9426994668197
{'rf__max_depth': 20, 'rf__max_features': 'log2', 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 200}


In [8]:
x_train_cluster_2 , x_test_cluster_2 , y_train_cluster_2 , y_test_cluster_2 = train_test_split(X_train_2,
                                                                                    y_train_2,test_size=0.2,random_state=42)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# Create Pipeline with feature selection and RandomForest
pipeline_rf = Pipeline([
    ('feature_selection', SelectFromModel(RandomForestRegressor())),
    ('rf', RandomForestRegressor())
])

# Define parameter grid for hyperparameter tuning for RandomForest
rf_params = {
    'rf__n_estimators': [50, 100, 200, 300],
    'rf__max_depth': [None, 5, 10, 20],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__max_features': ['auto', 'sqrt', 'log2']
}

# Perform GridSearchCV for RandomForest
grid_search_rf = GridSearchCV(pipeline_rf, rf_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV for RandomForest
grid_search_rf.fit(x_train_cluster_2, y_train_cluster_2)

# Get the best model for RandomForest
best_model_rf_2 = grid_search_rf.best_estimator_

# Get the selected features for RandomForest
selected_features_rf = x_train_cluster_2.columns[best_model_rf_2.named_steps['feature_selection'].get_support()]

# Evaluate the best model for RandomForest
mape_train_rf = np.mean(np.abs((y_train_cluster_2 - best_model_rf_2.predict(x_train_cluster_2)) / y_train_cluster_2)) * 100
mape_test_rf = np.mean(np.abs((y_test_cluster_2 - best_model_rf_2.predict(x_test_cluster_2)) / y_test_cluster_2)) * 100

print("Selected Features (RandomForest):", selected_features_rf)
print("Mean Absolute Percentage Error on Training set (RandomForest):", mape_train_rf)
print("Mean Absolute Percentage Error on Testing set (RandomForest):", mape_test_rf)

Selected Features (RandomForest): Index(['name', 'dish_liked'], dtype='object')
Mean Absolute Percentage Error on Training set (RandomForest): 0.6908083009801965
Mean Absolute Percentage Error on Testing set (RandomForest): 1.4624562257849252


In [11]:
# Predictions for Cluster 2 using Random Forest
y_train_pred_rf_cluster_2 = best_model_rf_2.predict(x_train_cluster_2)
y_test_pred_rf_cluster_2 = best_model_rf_2.predict(x_test_cluster_2)

# Calculate RMSE for Cluster 2 using Random Forest
rmse_train_rf_cluster_2 = np.sqrt(mean_squared_error(y_train_cluster_2, y_train_pred_rf_cluster_2))
rmse_test_rf_cluster_2 = np.sqrt(mean_squared_error(y_test_cluster_2, y_test_pred_rf_cluster_2))

# Mean Absolute Percentage Error for Cluster 2 using Random Forest
mape_train_rf_cluster_2 = np.mean(np.abs((y_train_cluster_2 - y_train_pred_rf_cluster_2) / y_train_cluster_2)) * 100
mape_test_rf_cluster_2 = np.mean(np.abs((y_test_cluster_2 - y_test_pred_rf_cluster_2) / y_test_cluster_2)) * 100

print("Selected Features (Random Forest) for Cluster 2:", selected_features_rf)
print("Mean Absolute Percentage Error on Training set (Random Forest) for Cluster 2:", mape_train_rf_cluster_2)
print("Mean Absolute Percentage Error on Testing set (Random Forest) for Cluster 2:", mape_test_rf_cluster_2)
print("Root Mean Squared Error on Training set (Random Forest) for Cluster 2:", rmse_train_rf_cluster_2)
print("Root Mean Squared Error on Testing set (Random Forest) for Cluster 2:", rmse_test_rf_cluster_2)
print(grid_search_rf.best_params_)


Selected Features (Random Forest) for Cluster 2: Index(['name', 'dish_liked'], dtype='object')
Mean Absolute Percentage Error on Training set (Random Forest) for Cluster 2: 0.6908083009801965
Mean Absolute Percentage Error on Testing set (Random Forest) for Cluster 2: 1.4624562257849252
Root Mean Squared Error on Training set (Random Forest) for Cluster 2: 14.955298918984171
Root Mean Squared Error on Testing set (Random Forest) for Cluster 2: 34.976263868466035
{'rf__max_depth': None, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}


In [20]:
selected_features_rf

Index(['name', 'dish_liked'], dtype='object')

In [21]:
from sklearn.preprocessing import MinMaxScaler

In [22]:
new_preds = clf.predict(X_test)

# Define best models for each cluster
best_model_cluster_1 =best_model_rf_1
best_model_cluster_2 = best_model_rf_2

# Assign cluster labels to X_test
X_test['cluster_label'] = new_preds

# Filter X_test for each cluster
X_test_1 = X_test[X_test['cluster_label'] == -1]
X_test_2 = X_test[X_test['cluster_label'] == 1]

# Get the indexes for each cluster
indexes_1 = X_test_1.index
indexes_2 = X_test_2.index

# Filter y_test for each cluster
y_test_1 = y_test.loc[indexes_1]
y_test_2 = y_test.loc[indexes_2]

# Predictions for each cluster using the corresponding best model
y_test_pred_cluster_1 = best_model_cluster_1.predict(X_test_1)
y_test_pred_cluster_2 = best_model_cluster_2.predict(X_test_2)

# Calculate Mean Absolute Percentage Error (MAPE) for cluster 1
mape_cluster_1 = np.mean(np.abs((y_test_1 - y_test_pred_cluster_1) / y_test_1)) * 100

# Calculate Mean Absolute Percentage Error (MAPE) for cluster 2
mape_cluster_2 = np.mean(np.abs((y_test_2 - y_test_pred_cluster_2) / y_test_2)) * 100

# Print MAPE for each cluster
print("Mean Absolute Percentage Error for Cluster 1:", mape_cluster_1)
print("Mean Absolute Percentage Error for Cluster 2:", mape_cluster_2)

Mean Absolute Percentage Error for Cluster 1: 4.683200107291389
Mean Absolute Percentage Error for Cluster 2: 7.50545874908213


In [23]:
import math

In [24]:
# Calculate Mean Squared Error (MSE) for cluster 1
mse_cluster_1 = mean_squared_error(y_test_1, y_test_pred_cluster_1)

# Calculate Mean Squared Error (MSE) for cluster 2
mse_cluster_2 = mean_squared_error(y_test_2, y_test_pred_cluster_2)

# Calculate Root Mean Squared Error (RMSE) for cluster 1
rmse_cluster_1 = math.sqrt(mse_cluster_1)

# Calculate Root Mean Squared Error (RMSE) for cluster 2
rmse_cluster_2 = math.sqrt(mse_cluster_2)

# Print MSE and RMSE for each cluster
print("Mean Squared Error for Cluster 1:", mse_cluster_1)
print("Root Mean Squared Error for Cluster 1:", rmse_cluster_1)
print("Mean Squared Error for Cluster 2:", mse_cluster_2)
print("Root Mean Squared Error for Cluster 2:", rmse_cluster_2)

Mean Squared Error for Cluster 1: 43366.703369468276
Root Mean Squared Error for Cluster 1: 208.2467367558692
Mean Squared Error for Cluster 2: 9772.36851354447
Root Mean Squared Error for Cluster 2: 98.85529077163483


In [25]:
# Combine predictions and actual values for both clusters
y_test_combined = pd.concat([y_test_1, y_test_2], axis=0)

# Combine predictions for both clusters
y_pred_combined = np.concatenate([y_test_pred_cluster_1, y_test_pred_cluster_2])

# Calculate MAPE for combined predictions
mape_combined = np.mean(np.abs((y_test_combined - y_pred_combined) / y_test_combined)) * 100

# Print MAPE for combined predictions
print("Mean Absolute Percentage Error for Combined Predictions:", mape_combined)

Mean Absolute Percentage Error for Combined Predictions: 6.988537258162133


In [26]:
# Calculate Mean Squared Error (MSE) for combined predictions
mse_combined = mean_squared_error(y_test_combined, y_pred_combined)

# Calculate Root Mean Squared Error (RMSE) for combined predictions
rmse_combined = math.sqrt(mse_combined)

# Print MSE and RMSE for combined predictions
print("Mean Squared Error for Combined Predictions:", mse_combined)
print("Root Mean Squared Error for Combined Predictions:", rmse_combined)

Mean Squared Error for Combined Predictions: 15925.466391622414
Root Mean Squared Error for Combined Predictions: 126.19614253859908
